In [1]:
import pandas as pd

## For IPL

In [27]:
def resFun(val1: str, val2: str):
    if val1 == val2:
        return 1
    else:
        return 0

In [64]:
def Ipl_winProb(pklFile:str):
    sdf_1 = pd.read_pickle(pklFile)
    sdf_2 = sdf_1[sdf_1['ininings'] == 2]
    sdf_3 = sdf_1[sdf_1['ininings'] == 1] 
    sdf_3 = sdf_3.groupby('Match_id').last()['runs_cumsum'].reset_index()
    sdf_4 = sdf_2.merge(sdf_3, on='Match_id', how='inner')
    sdf_4 = sdf_4.drop(columns=['season', 'dates','ininings'])
    sdf_4 = sdf_4.rename(columns={'ininings.team': 'team_batting', 
                              'runs.total':'runs', 'runs_cumsum_x':'runs_cum',
                             'ininings.overs.over':'overs','runs_cumsum_y':'target'})
    teams = ['Royal Challengers Bangalore','Kolkata Knight Riders','Sunrisers Hyderabad',
         'Mumbai Indians','Chennai Super Kings','Rajasthan Royals','Delhi Capitals',
         'Punjab Kings','Gujarat Titans','Lucknow Super Giants']
#     print(len(sdf_4['Match_id'].unique()))
    sdf_4['team_batting'] = sdf_4['team_batting'].str.replace('Kings XI Punjab', 'Punjab Kings')
    sdf_4['team_bowling'] = sdf_4['team_bowling'].str.replace('Kings XI Punjab', 'Punjab Kings')
    sdf_4['team_batting'] = sdf_4['team_batting'].str.replace('Delhi Daredevils', 'Delhi Capitals')
    sdf_4['team_bowling'] = sdf_4['team_bowling'].str.replace('Delhi Daredevils', 'Delhi Capitals')
    sdf_4['team_batting'] = sdf_4['team_batting'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
    sdf_4['team_bowling'] = sdf_4['team_bowling'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
#     print(len(sdf_4['Match_id'].unique()))
    sdf_4 = sdf_4[sdf_4['team_batting'].isin(teams)]
    sdf_4 = sdf_4[sdf_4['team_bowling'].isin(teams)]
#     print(len(sdf_4['Match_id'].unique()))
    sdf_4['overs'] = sdf_4['overs'].apply(lambda x: (x+1))
    sdf_4['balls'] = sdf_4.groupby('Match_id').cumcount()
    sdf_4['balls'] = sdf_4['balls'].apply(lambda x: (x + 1))
    result =[]
    val1 = sdf_4['team_batting'].values.tolist()
    val2 = sdf_4['winner'].values.tolist()
    if len(val1) == len(val2):
        for i in range(len(val1)):
            res =resFun(val1[i], val2[i])
            result.append(res)
    #  result 0 means bowling team wins and 1 means batting team wins
    sdf_4['Result'] = result
    sdf_4['runs_left'] = sdf_4['target'] - sdf_4['runs_cum']
    sdf_4['ball_left'] = 120 - sdf_4['balls']
    sdf_4['wicket_left'] = 10 - sdf_4['wicket_cumsum']
    sdf_4['ball_left'] = sdf_4['ball_left'].apply(lambda x: None if x <= 0 else int(x) )
    sdf_4 = sdf_4.dropna()
    sdf_4['ball_left'] = sdf_4['ball_left'].astype('int64')
#     print(len(sdf_4['Match_id'].unique()))
    sdf_4['crr'] = (((sdf_4['target'] - sdf_4['runs_left'])*6) / (120 - sdf_4['ball_left']))
    sdf_4['crr'] = sdf_4['crr'].apply(lambda x: round(x,2))
    sdf_4['rrr'] = ((sdf_4['runs_left']*6) / sdf_4['ball_left'])
    sdf_4['rrr'] = sdf_4['rrr'].apply(lambda x: round(x,2))
    sdf_4 = sdf_4.drop(columns=['Match_id','winner','runs','runs_cum', 'wicket_cumsum','overs','balls'])
    sdf_5 = sdf_4[['team_batting','team_bowling','city','runs_left','ball_left','wicket_left','target','crr','rrr','Result']]
    return sdf_5

In [65]:
# IplWinProb =Ipl_winProb('../ipl_DataClean/IPL_Data_Cleaning_cached_df')

In [66]:
# IplWinProb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89432 entries, 0 to 111835
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   team_batting  89432 non-null  object 
 1   team_bowling  89432 non-null  object 
 2   city          89432 non-null  object 
 3   runs_left     89432 non-null  int64  
 4   ball_left     89432 non-null  int64  
 5   wicket_left   89432 non-null  int64  
 6   target        89432 non-null  int64  
 7   crr           89432 non-null  float64
 8   rrr           89432 non-null  float64
 9   Result        89432 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 7.5+ MB


In [38]:
# IplWinProb.to_pickle('../ipl_DataClean/IplWinProb_Data_cached_df')

In [41]:
# pd.read_pickle('../ipl_DataClean/IplWinProb_Data_cached_df')

## For T20

In [58]:
def T20_winProb(pklFile:str):
    sdf_1 = pd.read_pickle(pklFile)
    sdf_2 = sdf_1[sdf_1['ininings'] == 2]
    sdf_3 = sdf_1[sdf_1['ininings'] == 1] 
    sdf_3 = sdf_3.groupby('Match_id').last()['runs_cumsum'].reset_index()
    sdf_4 = sdf_2.merge(sdf_3, on='Match_id', how='inner')
    sdf_4 = sdf_4.drop(columns=['dates','ininings'])
    sdf_4 = sdf_4.rename(columns={'ininings.team': 'team_batting', 
                              'runs.total':'runs', 'runs_cumsum_x':'runs_cum',
                             'ininings.overs.over':'overs','runs_cumsum_y':'target'})
    teams = ['Australia','Sri Lanka','New Zealand','Bangladesh','India','Ireland','Scotland','Netherlands',
             'West Indies','Pakistan','Zimbabwe','South Africa','England','Afghanistan']
#     print(len(sdf_4['Match_id'].unique()))
    sdf_4 = sdf_4[sdf_4['team_batting'].isin(teams)]
    sdf_4 = sdf_4[sdf_4['team_bowling'].isin(teams)]
#     print(len(sdf_4['Match_id'].unique()))
    sdf_4['overs'] = sdf_4['overs'].apply(lambda x: (x+1))
    sdf_4['balls'] = sdf_4.groupby('Match_id').cumcount()
    sdf_4['balls'] = sdf_4['balls'].apply(lambda x: (x + 1))
    result =[]
    val1 = sdf_4['team_batting'].values.tolist()
    val2 = sdf_4['winner'].values.tolist()
    if len(val1) == len(val2):
        for i in range(len(val1)):
            res =resFun(val1[i], val2[i])
            result.append(res)
    #  result 0 means bowling team wins and 1 means batting team wins
    sdf_4['Result'] = result
    sdf_4['runs_left'] = sdf_4['target'] - sdf_4['runs_cum']
    sdf_4['ball_left'] = 120 - sdf_4['balls']
    sdf_4['wicket_left'] = 10 - sdf_4['wicket_cumsum']
    sdf_4['ball_left'] = sdf_4['ball_left'].apply(lambda x: None if x <= 0 else int(x) )
    sdf_4 = sdf_4.dropna()
#     print(len(sdf_4['Match_id'].unique()))
    sdf_4['ball_left'] = sdf_4['ball_left'].astype('int64')
    sdf_4['crr'] = (((sdf_4['target'] - sdf_4['runs_left'])*6) / (120 - sdf_4['ball_left']))
    sdf_4['crr'] = sdf_4['crr'].apply(lambda x: round(x,2))
    sdf_4['rrr'] = ((sdf_4['runs_left']*6) / sdf_4['ball_left'])
    sdf_4['rrr'] = sdf_4['rrr'].apply(lambda x: round(x,2))
    sdf_4 = sdf_4.drop(columns=['Match_id','winner','runs','runs_cum', 'wicket_cumsum','overs','balls'])
    sdf_5 = sdf_4[['team_batting','team_bowling','city','runs_left','ball_left','wicket_left','target','crr','rrr','Result']]
    return sdf_5

In [59]:
# T20winProd = T20_winProb('../t20_DataClean/t20_Data_Cleaning_cached_df')

2576
1438
1240


In [60]:
# T20winProd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135071 entries, 125 to 276723
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   team_batting  135071 non-null  object 
 1   team_bowling  135071 non-null  object 
 2   city          135071 non-null  object 
 3   runs_left     135071 non-null  int64  
 4   ball_left     135071 non-null  int64  
 5   wicket_left   135071 non-null  int64  
 6   target        135071 non-null  int64  
 7   crr           135071 non-null  float64
 8   rrr           135071 non-null  float64
 9   Result        135071 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 11.3+ MB


In [67]:
# T20winProd.to_pickle('../t20_DataClean/T20WinProb_Data_cached_df')

In [69]:
# pd.read_pickle('../t20_DataClean/T20WinProb_Data_cached_df')